In [312]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

path = 'Housing.csv'

data = pd.read_csv(path)
index = [i for i in range(data.shape[0])]
np.random.shuffle(index)
data = data.set_index([index]).sort_index()
y = data["PRICE"]
X = data[data.columns.difference(['PRICE'])]
# display(X.describe())
k = 5
X -= X.mean(axis=0, skipna = True)
y -= y.mean()
X_shuffled = {}
y_shuffled = {}
rmse_train = []
rmse_test = []
for i in range(0,k):
    X_shuffled[i] = X[i::k]
    y_shuffled[i] = y[i::k]

In [313]:
def get_next_train_valid(X_shuffled, y_shuffled, itr):
    X_train = pd.DataFrame()
    y_train = pd.DataFrame()
    X_valid = pd.DataFrame()
    y_valid = pd.DataFrame()
    for i in range(0,k):
        if i != itr:
            X_train = X_train.append(X_shuffled.get(i))
            y_train = pd.concat([y_train, y_shuffled.get(i)])
        else:
            X_valid = X_shuffled.get(i)
            y_valid = y_shuffled.get(i)
    return X_train, y_train, X_valid, y_valid

In [314]:
def train(X_train, y_train, vlambda):
    C = np.dot(X_train.transpose(), X_train) + vlambda*np.matrix(np.identity(np.shape(X_train)[1]))
    model_weights = np.dot(np.dot(np.linalg.inv(C),X_train.transpose()),y_train)
    model_intercept = y_train.mean()
    return model_weights, model_intercept

In [315]:
def predict(X_valid, model_weights, model_intercept):
    y_predict = np.dot(X_valid,model_weights) + model_intercept[0]
    return y_predict

In [316]:
def RMSE():
    for vlambda in range(0,100):
        splitData(k);
        cur_rmse_train = 0
        cur_rmse_test = 0
        for i in range(0,k):
            X_train, y_train, X_valid, y_valid = get_next_train_valid(X_shuffled, y_shuffled, i)
            model_weights, model_intercept = train(X_train, y_train, vlambda)
            y_predict = predict(X_valid, model_weights, model_intercept)
            cur_rmse_train += np.sqrt(((pd.DataFrame(y_predict.flatten().transpose()).values - y_valid.values)**2).mean())
            cur_rmse_test += np.sqrt(((pd.DataFrame(predict(X_train, model_weights, model_intercept).flatten().transpose()).values - y_train.values)**2).mean())
        rmse_train.append(cur_rmse_train/k);
        rmse_test.append(cur_rmse_test/k);


In [317]:
RMSE()